# Run before playing

## Card class definition

In [ ]:
fig_str = {9: '10', 10: ' J', 11: ' Q', 12: ' K'}
fig_uni = {9: 'A', 10: 'B', 11: 'D', 12: 'E'}
for idx in range(9):
  fig_str[idx] = ' ' + str(idx + 1)
  fig_uni[idx] = str(idx + 1)

class Figure():
  def __init__(self, idx:int):
    self.idx = idx
    self.uni = fig_uni[idx] if idx<13 else ''

  def __str__(self):
    return fig_str[self.idx] if self.idx<13 else str(self.idx + 1)

In [ ]:
col_str = {0: '♠', 1: '♡', 2: '♣', 3: '♢'}
col_uni = {0: 'A', 1: 'B', 2: 'D', 3: 'C'}

class Color():
  def __init__(self, idx:int):
    self.idx = idx
    self.uni = col_uni[idx] if idx<4 else ''
  
  def __str__(self):
    return col_str[self.idx] if self.idx<4 else str(self.idx + 1)

In [ ]:
class Card():
  def __init__(self, idx, figure_size = 13):
    self.idx = idx
    self.figure = Figure(idx%figure_size) 
    self.color = Color(idx//figure_size)
    self.uni = '1F0' + self.color.uni + self.figure.uni if self.figure.uni and self.color.uni else '1F0A0'
    self.known = True

  def __str__(self):
    if self.known:
      return f'{chr(int(self.uni, 16))} {self.color}{self.figure}'
    else:
      return '🂠 ? ?'
  
  def __eq__(self, card):
    return card.idx == self.idx

## Util class definition

In [ ]:
class Stack(list):
  def __init__(self, group, empty_str):
    super().__init__()
    self.group = group
    self.empty_str = empty_str
    self.backup = []
  
  def init(self, other):
    self.clear()
    self += other
    self.hide()
  
  def move(self, other, depth = 1):
    self += other[-depth:]
    del other[-depth:]
    if other.group=='Deck':
      self[-1].known = True
    elif other:
      other[-1].known = True
  
  def save(self):
    self.backup = [card for card in self]
  
  def load(self):
    self.init(self.backup)
  
  def hide(self):
    for card in self:
      card.known = False

  def get_unknown(self):
    unknown = 0
    for card in self:
      if not card.known:
        unknown += 1
    return unknown

  def __str__(self):
    if self:
      out = str(self[-1])
    else:
      out = self.empty_str
    return f'{out}'

In [ ]:
class Next():
  def __init__(self, max_lenght = 0):
    self.ask = {}
    self.move = []
    self.lenght = 3 + 2 + 5 + 4 + 5 + 2
    self.move_per_line = (max_lenght+2) // self.lenght
    self.auto = False
  
  def add(self, stack_before, stack_after, depth = 1):
    self.move.append((stack_before, stack_after, depth))
  
  def asking(self, stack_after, card_idx_1, card_idx_2 = None):
    for card_idx in [card_idx_1, card_idx_2]:
      if card_idx != None:
        if card_idx not in self.ask:
          self.ask[card_idx] = []
        self.ask[card_idx].append(stack_after)
  
  def reply(self, stack_before, card_idx, depth = 1):
    for stack_after in self.ask[card_idx]:
      self.add(stack_before, stack_after, depth)
  
  def do(self, move_idx):
    turn = 0
    out = self.str_move(move_idx)
    stack_before, stack_after, depth = self.move[move_idx]
    stack_after.move(stack_before, depth)
    if stack_after.group == 'Deck':
      stack_after.hide()
      stack_after.reverse()
      stack_before.move(stack_after)
      turn = 1
    return out, turn

  def str_move(self, move_idx):
    stack_before, stack_after, depth = self.move[move_idx]
    if depth > 1 and stack_before:
      before = stack_before[-depth]
    else:
      before = stack_before
    return f"{f'{move_idx}':>3}: {f'{before}':>5} -> {f'{stack_after}':<5}"

  def sort(self):
    sorting = []
    for stack_before, stack_after, depth in self.move:
      if stack_before.group == 'Deck' or stack_after.group == 'Deck' :
        sorting_idx = 0
      elif stack_after.group == 'Sort':
        sorting_idx = 2
      elif stack_before.group == 'Hand':
        sorting_idx = 3
      elif stack_before.group == 'Sort':
        sorting_idx = 1001
      elif stack_before.get_unknown():
        if stack_before[-depth-1].known:
          sorting_idx = 1002
        else:
          sorting_idx = 1 + (50-stack_before.get_unknown())/50
      else:
        sorting_idx = 1003 if stack_before[0].figure.idx == 12 else 100
      while sorting_idx in sorting:
        sorting_idx += 0.01
      sorting.append(sorting_idx)
    self.move = [move_idx for _, move_idx in sorted(zip(sorting, self.move))]

  def __str__(self):
    output = ' 42: Auto fill\n' if self.auto else ''
    for move_idx in range(len(self.move)):
      output += self.str_move(move_idx)
      output += '  ' if (move_idx+1) % self.move_per_line else '\n'
    return output + f"\n {'69: Abord':<21}\n"

  def __bool__(self):
    return bool(self.move)

## Game class

In [ ]:
import random
import numpy as np
from IPython.display import clear_output 
import sys
import time

class Solitaire():
  def __init__(self, 
               board_size = 8,
               color_size = 4,
               figure_size = 13,
               seed = None):

    self.deck_size = color_size * figure_size
    self.board_size = board_size
    self.color_size = color_size
    self.figure_size = figure_size

    if seed == None:
      seed = random.randrange(sys.maxsize)

    self.log = 'SOLITAIRE'
    self.next_move = []

    self.deck = Stack('Deck', '▯ < <')
    self.play(seed)


  def reset(self, seed):
    self.deck.init([Card(idx, self.figure_size) for idx in range(self.deck_size)])
    self.hand = Stack('Hand', '▯')
    self.sort = [Stack('Sort', str(Color(idx))) for idx in range(self.color_size)]
    self.board = [Stack('Board', '') for _ in range(self.board_size)]

    random.seed(seed)
    random.shuffle(self.deck)

    for idx, stack in enumerate(self.board):
      stack.move(self.deck, idx + 1)


  def get_next(self, max_lenght):
      ## find next possibles moves
      next = Next(max_lenght)

      # possible drawing or refill of the deck
      if self.deck:
        next.add(self.deck, self.hand)
      elif self.hand:
        next.add(self.hand, self.deck, len(self.hand))
      
      # possible cards movable to board
      ask_king = True
      for stack in self.board:
        if stack:
          if stack[-1].figure.idx != 0:
            idx_1 = ((stack[-1].idx - 1) + self.figure_size) % self.deck_size
            idx_2 = (idx_1 + 2*self.figure_size) % self.deck_size
            next.asking(stack, idx_1, idx_2)
        elif ask_king:
          ask_king = False
          for color in range(self.color_size):
            next.asking(stack, (color + 1)*self.figure_size - 1)
      
      # possible stack of cards movable from board to the board
      for stack in self.board:
        if stack:
          for height, card in enumerate(stack):
            if card.known and card.idx in next.ask and height != len(stack) - 1:
              next.reply(stack, card.idx, len(stack) - height)

      # possible shorted cards movable and possible cards shortable
      for idx, color in enumerate(self.sort):
        next.asking(color, idx*self.figure_size + len(color))
        if color:
          if color[-1].idx in next.ask:
            next.reply(color, color[-1].idx)

      # possible drawn cards and top card shortable or movable
      if self.hand:
        if self.hand[-1].idx in next.ask:
          next.reply(self.hand, self.hand[-1].idx)
      for stack in self.board:
        if stack:
          if stack[-1].idx in next.ask:
              next.reply(stack, stack[-1].idx)
      next.sort()
      return next


  def is_win(self):
    for color in self.sort:
      if len(color) != self.figure_size:
        return False
    return True

  def is_auto_fill(self):
    for stack in self.board:
      if stack.get_unknown() > 0:
        return False
    return True

  def play(self, seed):
    self.reset(seed)
    step = 0
    turn = 0
    move = 'begining'
    state = 'PLAY'

    while 1:
      next = self.get_next(7*(self.board_size + 2))

      if not next:
        state = 'LOSE'
      if self.is_win():
        state = 'WIN'
      if self.is_auto_fill():
        next.auto = True
      self.log = f"seed: {seed:>18}  move {step:<3}: {move:<8}  deck turn: {turn:<3}  ETA: {state}"

      output = str(self)
      clear_output()
      print(output)
      print(next)

      if state == 'PLAY':
        move = None
        while move == None:
          move_idx = input('Your move: ')
          if not move_idx:
            move_idx = '0'
          if move_idx.isdigit():
            move_idx = int(move_idx)
            if move_idx == 42 and next.auto:
              state = 'AUTO'
              move = 'auto'
            elif move_idx == 69:
              state = 'LOST'
              move = 'abord'
            elif move_idx in range(len(next.move)):
              move, add_turn = next.do(move_idx)
              turn += add_turn
              step += 1
            else:
              print('Please input a possible move')
          else:
            print('Please input digits only')
      
      if state == 'AUTO':
        if len(next.move) == 1:
          idx_move = 0
        idx_move = 1 if next.move[1][1].group == 'Sort' or next.move[1][0].group == 'Hand' else 0
        move, add_turn = next.do(idx_move)
        turn += add_turn
        step += 1
        time.sleep(0.5)

      if state in ['WIN', 'LOST']:
        restart = input('[N] : New game or R : Restart the same ?')
        if restart in ['R', 'Restart', 'r', 'restart', 're', 'Re', 'Restart the same', 'restart the same']:
          seed = seed
          move = 'restarting'
        elif restart.isdigit():
          seed = int(restart)
          move = 'using seed'
        else:
          seed = random.randrange(sys.maxsize)
          move = 'new game +'
        self.reset(seed)
        step = 0
        turn = 0
        state = 'PLAY'


  def __str__(self):
    show = [[' ' for column in range(self.board_size + 2)] for line in range(self.board_size + self.figure_size - 1)]

    for column, stack in enumerate(self.board):
      for line, card in enumerate(stack):
        show[line][column] = card

    for line, color in enumerate(self.sort):
      show[line][-1] = color
    
    show[-2][-1] = self.deck
    show[-1][-1] = self.hand
    
    output =''
    output += self.log + '\n\n'
    for line in show:
      for card in line:
        output += f"{f'{card}':<7}\t"
      output+='\n'
    return output

# The game

In [ ]:
Solitaire()

seed: 2846411226685922498  move 136:   0: 🃎 ♢ K -> 🃍 ♢ Q  deck turn: 16   ETA: WIN

       	       	       	       	       	       	       	       	       	🂮 ♠ K  	
       	       	       	       	       	       	       	       	       	🂾 ♡ K  	
       	       	       	       	       	       	       	       	       	🃞 ♣ K  	
       	       	       	       	       	       	       	       	       	🃎 ♢ K  	
       	       	       	       	       	       	       	       	       	       	
       	       	       	       	       	       	       	       	       	       	
       	       	       	       	       	       	       	       	       	       	
       	       	       	       	       	       	       	       	       	       	
       	       	       	       	       	       	       	       	       	       	
       	       	       	       	       	       	       	       	       	       	
       	       	       	       	       	       	       	       	       	       	
       	       	       	 